In [ ]:
# Import necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

# Step 1: Define paths and hyperparameters
dataset_path = "C:/Users/kushal dixit\Downloads\dataset\data"  
output_plot = "plot.png"
output_model = "fmd_model.h5"

INIT_LR = 1e-4
EPOCHS = 20
BS = 32

# Step 2: Load images and labels
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path))
print(f"[INFO] Total images found: {len(imagePaths)}")

data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

# Convert to numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# Step 3: One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Step 4: Split data
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

# Step 5: Data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Step 6: Load MobileNetV2 model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Step 7: Build head model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

# Step 8: Freeze base model layers
for layer in baseModel.layers:
    layer.trainable = False

# Step 9: Compile model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Step 10: Train model
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# Step 11: Evaluate model
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

# Step 12: Save model
print("[INFO] saving mask detector model...")
model.save(output_model, save_format="h5")

# Step 13: Plot training loss/accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(output_plot)


<>:19: SyntaxWarning: invalid escape sequence '\D'
<>:19: SyntaxWarning: invalid escape sequence '\D'
C:\Users\kushal dixit\AppData\Local\Temp\ipykernel_15996\179698097.py:19: SyntaxWarning: invalid escape sequence '\D'
  dataset_path = "C:/Users/kushal dixit\Downloads\dataset\data"


[INFO] loading images...
[INFO] Total images found: 7553


c:\Users\kushal dixit\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
C:\Users\kushal dixit\AppData\Local\Temp\ipykernel_15996\179698097.py:66: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))


[INFO] compiling model...
[INFO] training head...


c:\Users\kushal dixit\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
c:\Users\kushal dixit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.8060 - loss: 0.4761 - val_accuracy: 0.9735 - val_loss: 0.0961
Epoch 2/20


c:\Users\kushal dixit\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


188/188 ━━━━━━━━━━━━━━━━━━━━ 26s 138ms/step - accuracy: 1.0000 - loss: 0.0941 - val_accuracy: 0.9749 - val_loss: 0.0954
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.9758 - loss: 0.0987 - val_accuracy: 0.9788 - val_loss: 0.0617
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 27s 140ms/step - accuracy: 0.9688 - loss: 0.1038 - val_accuracy: 0.9788 - val_loss: 0.0616
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.9759 - loss: 0.0773 - val_accuracy: 0.9808 - val_loss: 0.0527
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 26s 135ms/step - accuracy: 1.0000 - loss: 0.0129 - val_accuracy: 0.9815 - val_loss: 0.0531
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.9811 - loss: 0.0611 - val_accuracy: 0.9815 - val_loss: 0.0550
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 25s 131ms/step - accuracy: 1.0000 - loss: 0.0155 - val_accuracy: 0.9815 - val_loss: 0.0551
Epoch 9/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.9845 - loss: 0.0511 - val_accurac

In [ ]:
model.save(r"C:/Users/kushal dixit/Downloads/fmd_model.h5")
